In [23]:
%pip install pandas
%pip install duckdb
import pandas as pd
import duckdb

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\adria\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\adria\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [24]:
# Criar conexão in-memory
con = duckdb.connect()

# Ler CSV diretamente sem RAM alta
con.execute("""
    CREATE TABLE infracoes AS
    SELECT * FROM read_csv_auto('../../dados/infracoes/combined_output.csv', delim=';', header=True, ignore_errors=True)
""")

In [25]:
con.execute("SELECT * FROM infracoes LIMIT 1000").fetchdf()

,Número do Auto,Data da Infração (DD/MM/AAAA),Indicador de Abordagem,Assinatura do Auto,Indicador Veiculo Estrangeiro,Sentido Trafego,UF Placa,UF Infração,BR Infração,Km Infração,...,Fim Vigência Infração,Medição Infração,Descrição Especie Veículo,Descrição Marca Veículo,Hora Infração,Medição Considerada,Excesso Verificado,Qtd Infrações,Descrição Tipo Veículo,Descrição Modelo Veiculo
0,5bcd39ea93ac463,2019-01-06,C,N,S,C,-1,PR,369,132,...,None,Nenhuma,PASSAGEIRO,FORD,18,0.00,0.00,None,None,None
1,e038e26abee1b3f,2019-01-20,C,N,N,D,-1,RJ,356,169,...,None,Nenhuma,PASSAGEIRO,OUTRA,16,0.00,0.00,None,None,None
2,e4881fa47a05ae9,2019-01-20,C,N,N,D,-1,RJ,356,169,...,None,Nenhuma,PASSAGEIRO,OUTRA,16,0.00,0.00,None,None,None
3,d6c91fa499ca5fd,2019-01-04,C,N,N,C,GO,GO,50,286,...,None,Nenhuma,MISTO,OUTRA,9,0.00,0.00,None,None,None
4,8af020201f2bebf,2019-01-04,C,N,N,D,RS,PR,116,98,...,None,Nenhuma,CARGA,MERCEDES BENZ,3,0.00,0.00,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,7f2f051c7d48d0b,2019-01-16,C,N,N,D,MG,MG,40,538,...,None,Nenhuma,CARGA,MERCEDES BENZ,18,0.00,0.00,None,None,None
996,5108680b70fe996,2019-01-19,S,N,N,C,MG,MG,381,467,...,None,Nenhuma,PASSAGEIRO,FIAT,16,0.00,0.00,None,None,None
997,5085cc86523bb47,2019-01-19,S,N,N,C,MG,MG,381,467,...,None,Nenhuma,PASSAGEIRO,HONDA,16,0.00,0.00,None,None,None
998,c9d423927272766,2019-01-24,C,N,N,C,MG,MG,381,473,...,None,Nenhuma,CARGA,MERCEDES BENZ,9,0.00,0.00,None,None,None


In [26]:
# contanto valores de indicador de abordagem:
query = """
SELECT "Indicador de Abordagem", COUNT(*) AS total
FROM infracoes
GROUP BY "Indicador de Abordagem"
ORDER BY total DESC
"""
con.execute(query).fetchdf()

,Indicador de Abordagem,total
0,S,27883163
1,C,10818020


In [27]:
# contanto valores de Assinatura do Auto:
query = """
SELECT "Assinatura do Auto", COUNT(*) AS total
FROM infracoes
GROUP BY "Assinatura do Auto"
ORDER BY total DESC
"""
con.execute(query).fetchdf()

,Assinatura do Auto,total
0,N,35193899
1,N/I,3492051
2,S,15233


In [28]:
# contanto valores de Indicador Veiculo Estrangeiro:
query = """
SELECT "Indicador Veiculo Estrangeiro", COUNT(*) AS total
FROM infracoes
GROUP BY "Indicador Veiculo Estrangeiro"
ORDER BY total DESC
"""
con.execute(query).fetchdf()

,Indicador Veiculo Estrangeiro,total
0,N,35016016
1,BR,3422498
2,S,187525
3,99,59171
4,AR,5338
5,/,4812
6,PY,4225
7,None,744
8,UY,422
9,BO,397


In [31]:
# contanto valores de Sentido Trafego:
query = """
SELECT "Sentido Trafego", COUNT(*) AS total
FROM infracoes
GROUP BY "Sentido Trafego"
ORDER BY total DESC
"""
con.execute(query).fetchdf()

,Sentido Trafego,total
0,C,19600558
1,D,19100612


In [ ]:
# retirar valores com "/"" de Sentido Trafego
con.execute("""
CREATE OR REPLACE TABLE infracoes AS
SELECT *
FROM infracoes
WHERE "Sentido Trafego" IS NOT NULL
  AND "Sentido Trafego" != '/'
""")

In [10]:
# contanto valores de UF Placa:
query = """
SELECT "UF Placa", COUNT(*) AS total
FROM infracoes
GROUP BY "UF Placa"
ORDER BY total DESC
"""
con.execute(query).fetchdf()

,UF Placa,total
0,None,9940077
1,RJ,3442176
2,MG,3224394
3,SP,3104724
4,'-1,2553720
...,...,...
92,61,1
93,26,1
94,LH,1
95,3D,1


In [11]:
# contanto valores de UF Placa:
query = """
SELECT "UF Infração", COUNT(*) AS total
FROM infracoes
GROUP BY "UF Infração"
ORDER BY total DESC
"""
con.execute(query).fetchdf()

,UF Infração,total
0,RJ,6307476
1,MG,4242883
2,SP,4154427
3,BA,2734949
4,SC,2570493
5,PR,2369910
6,ES,2167100
7,RS,2013849
8,MS,1930532
9,GO,1813596


In [12]:
# contanto valores de BR Infração:
query = """
SELECT "BR Infração", COUNT(*) AS total
FROM infracoes
GROUP BY "BR Infração"
ORDER BY total DESC
"""
con.execute(query).fetchdf()

,BR Infração,total
0,101,8912055
1,116,7543079
2,40,2382227
3,163,2370547
4,381,1602921
...,...,...
135,498,2
136,377,1
137,404,1
138,610,1


In [13]:
# contanto valores de Km Infração:
query = """
SELECT "Km Infração", COUNT(*) AS total
FROM infracoes
GROUP BY "Km Infração"
ORDER BY total DESC
"""
con.execute(query).fetchdf()

,Km Infração,total
0,1,480663
1,329,449180
2,2,431796
3,323,393569
4,332,342580
...,...,...
1278,1264,1
1279,1492,1
1280,6074,1
1281,5986,1


In [14]:
# contanto valores de Código da Infração:
query = """
SELECT "Código da Infração", COUNT(*) AS total
FROM infracoes
GROUP BY "Código da Infração"
ORDER BY total DESC
"""
con.execute(query).fetchdf()

,Código da Infração,total
0,74550,17811488
1,74630,2979610
2,59670,1411404
3,65992,1172062
4,51851,922553
...,...,...
390,51002,4
391,72002,3
392,54360,2
393,51003,2


In [15]:
# contanto valores de Medição Infração:
query = """
SELECT "Medição Infração", COUNT(*) AS total
FROM infracoes
GROUP BY "Medição Infração"
ORDER BY total DESC
"""
con.execute(query).fetchdf()

,Medição Infração,total
0,Veiculo,19533558
1,Nenhuma,15468178
2,N/I,3470379
3,Peso,102138
4,Alcoolemia,86392
5,Dimensão,26468
6,PBT/PBTC,8254
7,PESO PBT/PBTC,4244
8,CMT GRAVISIMA,836
9,PESO CMT GRAVISIMA,434


In [16]:
# contanto valores de Descrição Especie Veículo:
query = """
SELECT "Descrição Especie Veículo", COUNT(*) AS total
FROM infracoes
GROUP BY "Descrição Especie Veículo"
ORDER BY total DESC
"""
con.execute(query).fetchdf()

,Descrição Especie Veículo,total
0,PASSAGEIRO,17479023
1,CARGA,6006076
2,'-1,4804523
3,-1,3027027
4,ESPECIAL,2531543
5,MISTO,2503538
6,TRACAO,1171124
7,TRAÇÃO,989107
8,Não informado,181150
9,Não encontrado,4812


In [17]:
# contanto valores de Descrição Marca Veículo:
query = """
SELECT "Descrição Marca Veículo", COUNT(*) AS total
FROM infracoes
GROUP BY "Descrição Marca Veículo"
ORDER BY total DESC
"""
con.execute(query).fetchdf()

,Descrição Marca Veículo,total
0,'-1,4666242
1,-1,2645998
2,VW,1860136
3,HONDA,1753007
4,FIAT,1309011
...,...,...
26943,DONGFENG,1
26944,SR BONANO,1
26945,HONDA/NX,1
26946,R/MORINI M JT,1


In [18]:
# contanto valores de Hora Infração:
query = """
SELECT "Hora Infração", COUNT(*) AS total
FROM infracoes
GROUP BY "Hora Infração"
ORDER BY total DESC
"""
con.execute(query).fetchdf()

,Hora Infração,total
0,10,3821568
1,16,3401478
2,11,3191990
3,9,3059946
4,15,3043319
5,17,2794431
6,14,2274450
7,8,2102764
8,12,1816205
9,13,1750200


In [19]:
# contanto valores de Medição Considerada:
query = """
SELECT "Medição Considerada", COUNT(*) AS total
FROM infracoes
GROUP BY "Medição Considerada"
ORDER BY total DESC
"""
con.execute(query).fetchdf()

,Medição Considerada,total
0,0,13251286
1,0.00,2664815
2,0.0,1474421
3,61,713080
4,81,638151
...,...,...
47193,97190.0,1
47194,73309.0,1
47195,46620.0,1
47196,23115.0,1


In [20]:
# contanto valores de Excesso Verificado:
query = """
SELECT "Excesso Verificado", COUNT(*) AS total
FROM infracoes
GROUP BY "Excesso Verificado"
ORDER BY total DESC
"""
con.execute(query).fetchdf()

,Excesso Verificado,total
0,0,12979102
1,0.00,2664886
2,1,1841001
3,0.0,1746679
4,2,1572884
...,...,...
31490,"3689,82",1
31491,"2414,75",1
31492,8182,1
31493,"12388,9",1
